5.1 LLM agents
==========
**Author:** Polina Tsvilodub

This sheet takes a closer look at more complex LLM-based systems and *LLM agents*. Specifically, we will use the package [`langchain`](https://python.langchain.com/v0.2/docs/tutorials/) and its extensions to build our own LLM systems and explore their functionality. The learning goals for this sheet are:

* understanding basics of langchain 
* trying out langchain agents and tools 
* understanding the basics of output processing 
* familiarization with basic handling of agent memory

## LangChain

The lecture discussed that modern LLMs can be viewed as building blocks of larger systems, be it for engineering or research purposes. In particular, one might want to use an LLM and make several calls (i.e., several inference passses) to it, and somehow use the predicted results together to complete one's task. 
 

[Disclaimer: Not sponsored by LangChain -- there are other very useful tools for doing such things, for instance, XXXX. This is just one popular example]

In [ ]:
#!pip install langchain==0.2.2 langchain-core==0.2.4 langchain-openai==0.1.7 wikipedia==1.4.0

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [26]:
# set some hyperparameters
temperature = 0.7
kwargs = {
    "max_tokens": 100,
}

ingredients = "cauliflower, tomatoes."

instructions_text_appetizer = "I have the following ingredients in my fridge: \n{ingredients}\n\nWhich Italian appetizer can I make for dinner with these ingredients?"

instructions_text_main = "I am planning to make the following appetizer: \n{appetizer}\n\nWhich Italian main course can I make for my dinner?"

instructions_text_dessert = "I have the following ingredients in my fridge: \n{ingredients}\n\nI am planning to make the following appetizer: \n{appetizer}\n\nThe, I will serve the following main course:\n{main_course}\n\Which Italian dessert could I make fitting my dinner menu?"


In [31]:
load_dotenv()

# instantiate model
model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=temperature,
    **kwargs   
) 

prompt_template_appetizer = PromptTemplate(
    template = instructions_text_appetizer,
    input_variables = ['ingredients'],
    # output_variables = ['appetizer'],
)
prompt_template_main = PromptTemplate(
    template = instructions_text_main,
    input_variables = ['appetizer'],
    # output_variables = ['main_course'],
)
prompt_template_dessert = PromptTemplate(
    template = instructions_text_appetizer,
    input_variables = ['ingredients', 'appetizer', 'main_course'],
)

# prompt_template = ChatPromptTemplate.from_messages(
#     [("system", "You are a helpful assistant."), ("user", instructions_text)]
# )

# format the few_shot prompt
# few_shot_prompt = FewShotPromptTemplate(
#     prefix=instructions_text, 
#     examples=selected_examples,
#     example_prompt=prompt_template, 
#     input_variables=[], 
#     suffix="Object category:",
#     example_separator="\n\n",
# )    
# print("Predicting objects")

appetizer_chain = prompt_template_appetizer | model | StrOutputParser()
# r = appetizer_chain.invoke({"ingredients": ingredients})
# print("appetizer chaon result: ", r)
composed_chain = {"appetizer": appetizer_chain} | prompt_template_main | model
composed_result = composed_chain.invoke({"ingredients": ingredients})
print("Result: ", composed_result)



You can make a classic Italian dish called "Spaghetti Carbonara." Here's how to make it:

Ingredients:
- 1 lb spaghetti
- 6 slices of pancetta or bacon, diced
- 2 cloves of garlic, minced
- 2 large eggs
- 1 cup grated Pecorino Romano cheese
- Salt and pepper to taste
- Chopped parsley for garnish

Instructions:
1. Cook the spaghetti according to package instructions until al dente.

### Agents

[React agent](https://react-lm.github.io/)

In [8]:
# same task with agent
from langchain import hub
import langchain_core
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent


# tools = [search, retriever_tool]

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.5)


# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
# just sets up a helpful assistant etc
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [9]:
agent = create_react_agent(llm, tools=[], prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=[], verbose=True)

agent_executor.invoke({"input": "Please help me come up with a three course Italian dinner menu. It should be vegetarian. I have cauliflower and tomatoes in my fridge."})



> Entering new AgentExecutor chain...
First, think about what dishes can be made using cauliflower and tomatoes in an Italian cuisine. Consider appetizers, main courses, and desserts.
Action: [Search for vegetarian Italian recipes using cauliflower and tomatoes]
Action Input: Vegetarian cauliflower and tomato Italian recipes[Search for vegetarian Italian recipes using cauliflower and tomatoes] is not a valid tool, try one of [].Let's try searching for recipes on a cooking website or app.
Action: [Search for vegetarian Italian recipes using cauliflower and tomatoes on a cooking website or app]
Action Input: Vegetarian cauliflower and tomato Italian recipes[Search for vegetarian Italian recipes using cauliflower and tomatoes on a cooking website or app] is not a valid tool, try one of [].Let's try searching for recipes on a popular search engine.
Action: [Search for vegetarian Italian recipes using cauliflower and tomatoes on a popular search engine]
Action Input: Vegetarian cauliflowe

{'input': 'Please help me come up with a three course Italian dinner menu. It should be vegetarian. I have cauliflower and tomatoes in my fridge.',
 'output': 'For the appetizer, I will make a roasted cauliflower and tomato bruschetta. For the main course, I will prepare a cauliflower and tomato pasta with garlic and olive oil. For dessert, I will serve a tomato and basil sorbet.'}

### LangChain agent with tools

Bla bla bla. Web search tool and math tool maybe.

In [11]:
from langchain.agents import load_tools
tools = load_tools(["wikipedia"], llm=llm) #[search, retriever_tool]

print('tools',  tools)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# from langchain import hub

# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# # just sets up a helpful assistant etc
# prompt.messages

from langchain.agents import create_tool_calling_agent, create_react_agent

agent_with_tools = create_react_agent(llm, tools=tools, prompt=prompt)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent_with_tools, tools=tools, verbose=True)

agent_executor.invoke({"input": "Please help me come up with a three course Italian dinner menu. It should be vegetarian. I have cauliflower and tomatoes in my fridge."})

tools [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/anaconda3/envs/understanding_llms/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))]


> Entering new AgentExecutor chain...
You can use Wikipedia to search for vegetarian Italian dishes that include cauliflower and tomatoes.
Action: wikipedia
Action Input: Vegetarian Italian dishes with cauliflower and tomatoesPage: List of vegetable dishes
Summary: This is a list of vegetable dishes, that includes dishes in which the main ingredient or one of the essential ingredients is a vegetable or vegetables.
In culinary terms, a vegetable is an edible plant or its part, intended for cooking or eating raw. Many vegetable-based dishes exist throughout the world.

Page: Fried cauliflower
Summary: Fried cauliflower is a popular dish in many cuisines of the Middle East, South Asia, Europe, and elsewhere. It may

{'input': 'Please help me come up with a three course Italian dinner menu. It should be vegetarian. I have cauliflower and tomatoes in my fridge.',
 'output': 'For the vegetarian Italian dinner menu, you can start with a Caprese salad with tomatoes and fresh mozzarella, followed by a main course of cauliflower risotto, and finish with a dessert of panna cotta with a tomato compote.'}

## Output parsing

https://python.langchain.com/v0.2/docs/how_to/structured_output/

## Memory handling & Cognitive agents